In [1]:
from helpers_min import get_table_simple, search_topk_hybrid_simple, get_ESMC, embed_sequence, get_sequence_embedding
import pandas as pd

# table_path = "C:\\Users\\eliaa\\Documents\\GitHub\\metalloprotein_search\\data\\BioLIP2\\BioLiP_metalscry_mapped.pkl"
# df = pd.read_pickle(table_path)
# df
# # 1) Build the lean table (expects df with columns: "sequence", "metal_residues")
# table = get_table_simple(df, embed=True, model_name="esmc_600m", device="cuda")
table_path_600 = "C:\\Users\\eliaa\\Documents\\GitHub\\metalloprotein_search\\data\\BioLIP2\\BioLiP_metalscry_mapped_embedded_600.pkl"
# table.to_pickle(table_path_600)
table = pd.read_pickle(table_path_600)

In [ ]:
query = table.iloc[59]
seq = query["sequence"]
model_name="esmc_600m"
client = get_ESMC(model_name=model_name)
outs = embed_sequence(client, seq) 
res_emb = outs.embeddings.detach().cpu().squeeze(0)          # (T, D) remove batch
seq_emb = get_sequence_embedding(outs).detach().cpu()        # (D,) or whatever you return
print(res_emb.shape)
print(seq_emb.shape)

In [ ]:
# 2) Run Top-K (global dot; residue-site 'sum' or 'mean')
hits = search_topk_hybrid_simple(
    seq, table, k=10, site_metric="sum", site_top_m=6,
    model_name="esmc_600m", device="cuda"
)
hits.drop(columns=["template_row"], inplace=True)
hits

,rank,idx,global_score,site_score,site_cols,top_query_indices,top_query_scores
0,1,2240,0.926855,15.977347,"[81, 91]","[11, 66, 15, 31, 166, 128]","[2.742351770401001, 2.718047618865967, 2.69215..."
1,2,3804,0.924346,22.256798,"[137, 199, 200]","[15, 11, 62, 22, 112, 54]","[3.904573440551758, 3.7685470581054688, 3.6591..."
2,3,3694,0.912163,0.000000,"[227, 232]","[0, 1, 2, 3, 4, 5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,4,4413,0.907823,23.075068,"[121, 122, 127]","[112, 94, 113, 3, 4, 56]","[3.9620890617370605, 3.917494297027588, 3.8804..."
4,5,5828,0.906473,15.841165,"[46, 47]","[59, 56, 113, 60, 173, 57]","[2.7938103675842285, 2.702932834625244, 2.6098..."
5,6,1519,0.905774,42.094952,"[21, 23, 26, 137, 138, 182]","[62, 15, 16, 3, 17, 54]","[7.368012428283691, 7.087433815002441, 7.02873..."
6,7,2424,0.903132,43.719070,"[121, 122, 127, 137, 141, 149]","[31, 30, 65, 62, 66, 44]","[7.42840576171875, 7.42374849319458, 7.2608809..."
7,8,1392,0.903076,0.000000,"[273, 276, 277]","[0, 1, 2, 3, 4, 5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,9,5729,0.903045,21.001930,"[199, 203, 217]","[62, 66, 96, 30, 31, 67]","[3.633970260620117, 3.5009825229644775, 3.4979..."
9,10,5446,0.902697,20.362484,"[139, 143, 161]","[62, 123, 66, 124, 34, 30]","[3.4319660663604736, 3.423248291015625, 3.4106..."


In [ ]:
# 2) Run Top-K (global dot; residue-site 'sum' or 'mean')
hits = search_topk_hybrid_simple(
    seq, table, k=10, site_metric="mean", site_top_m=6,
    model_name="esmc_600m", device="cuda"
)
hits.drop(columns=["template_row"], inplace=True)
hits

,rank,idx,global_score,site_score,site_cols,top_query_indices,top_query_scores
0,1,2240,0.926855,7.988674,"[81, 91]","[11, 66, 15, 31, 166, 128]","[1.3711758852005005, 1.3590238094329834, 1.346..."
1,2,3804,0.924346,7.418933,"[137, 199, 200]","[15, 11, 62, 22, 112, 54]","[1.3015245199203491, 1.256182312965393, 1.2197..."
2,3,3694,0.912163,0.000000,"[227, 232]","[0, 1, 2, 3, 4, 5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,4,4413,0.907823,7.691689,"[121, 122, 127]","[112, 94, 113, 3, 4, 56]","[1.3206963539123535, 1.3058314323425293, 1.293..."
4,5,5828,0.906473,7.920582,"[46, 47]","[59, 56, 113, 60, 173, 57]","[1.3969051837921143, 1.351466417312622, 1.3049..."
5,6,1519,0.905774,7.015825,"[21, 23, 26, 137, 138, 182]","[62, 15, 16, 3, 17, 54]","[1.2280020713806152, 1.1812390089035034, 1.171..."
6,7,2424,0.903132,7.286512,"[121, 122, 127, 137, 141, 149]","[31, 30, 65, 62, 66, 44]","[1.238067626953125, 1.2372914552688599, 1.2101..."
7,8,1392,0.903076,0.000000,"[273, 276, 277]","[0, 1, 2, 3, 4, 5]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
8,9,5729,0.903045,7.000643,"[199, 203, 217]","[62, 66, 96, 30, 31, 67]","[1.2113233804702759, 1.1669942140579224, 1.165..."
9,10,5446,0.902697,6.787495,"[139, 143, 161]","[62, 123, 66, 124, 34, 30]","[1.1439887285232544, 1.141082763671875, 1.1368..."
